In [1]:
import tifffile
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import os
import UTXCTregister as reg
import openpyxl
from pathlib import Path
import sys
# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the parent directory to sys.path
sys.path.append(parent_dir)
from AutoUTAligner import autoutaligner as alg

In [2]:
def read_sequence(folder_path):
    """
    Read a sequence of TIFF files in a folder as a 3D volume.
    
    Args:
    folder_path (str): Path to the folder containing TIFF files.

    Returns:
    numpy.ndarray: A 3D array where each slice corresponds to a TIFF file.
    """

    # List and sort the TIFF files
    tiff_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if (f.endswith('.tiff') or f.endswith('.tif'))])

    tiff_sequence = tifffile.TiffSequence(tiff_files)
    
    # Get the total number of TIFF files
    total_files = len(tiff_files)
    
    # Read each TIFF file and update progress
    volume = []
    with tqdm(total=total_files, desc="Progress") as pbar:
        for i, file_path in enumerate(tiff_files):
            slice_data = tifffile.imread(file_path)
            volume.append(slice_data)
            
            # Update progress
            pbar.update(1)
    
    return np.array(volume)

def write_sequence(folder_path, name, volume):
    """
    Save a 3D volume as a sequence of TIFF files in a folder.
    
    Args:
    folder_path (str): Path to the folder where TIFF files will be saved.
    name (str): Name of the TIFF files.
    volume (numpy.ndarray): A 3D array where each slice corresponds to an image.
    """

    folder_path = folder_path / name

    # Create the folder if it doesn't exist
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    # Save each slice as a TIFF file with progress bar
    with tqdm(total=volume.shape[0], desc="Saving") as pbar:
        for i in range(volume.shape[0]):
            tifffile.imwrite(f"{folder_path}/{name}_{i:04d}.tif", volume[i])
            pbar.update(1)
    
    print("Saving complete.")

In [3]:
database = pd.read_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv')

database = database.replace({np.nan: None})

In [4]:
database.columns

Index(['Id', 'Original', 'Eq', 'Frontal 90 Right', 'XCT padded', 'Binary',
       'Material Mask', 'Onlypores', 'UT', 'UT aligned', 'UT aligned amp',
       'UT registered', 'UT lente', 'UT aligned lente', 'UT aligned amp lente',
       'UT registered lente', 'Registration Parameters',
       'Registration Parameters lente', 'Dataset PatchvsVolfrac 3x3',
       'Dataset PatchvsPacth 3x3', 'Dataset PatchvsVolfrac lente 3x3',
       'Dataset PatchvsPacth lente 3x3', 'Dataset PatchvsVolfrac 5x5',
       'Dataset PatchvsPacth 5x5', 'Dataset PatchvsVolfrac lente 5x5',
       'Dataset PatchvsPacth lente 5x5', 'Dataset PatchvsVolfrac 7x7',
       'Dataset PatchvsPacth 7x7', 'Dataset PatchvsVolfrac lente 7x7',
       'Dataset PatchvsPacth lente 7x7'],
      dtype='object')

In [5]:
ids = database['Id'].values

frontals  = database['Frontal 90 Right'].values

xct_paddeds = database['XCT padded'].values

amps = database['UT aligned amp'].values

rfs = database['UT aligned'].values

rfs_og = database['UT'].values

rfs_og_lente = database['UT lente'].values

amps_lente = database['UT aligned amp lente'].values

rfs_lente = database['UT aligned lente'].values

registered_uts = database['UT registered'].values

registered_uts_lente = database['UT registered lente'].values

registration_parameters = database['Registration Parameters'].values

registration_parameters_lente = database['Registration Parameters lente'].values

In [6]:
# for i in range(len(ids)):

#     #utvsxct register
#     if registered_uts[i] == None: #if is not already registered

#         try:

#             print(f'Registering {ids[i]}')

#             ut_file = str(rfs[i])

#             ut_output_file = Path(ut_file[:-4] + '_registered.tif')

#             ut = tifffile.imread(amps[i])
#             ut = np.swapaxes(ut, 0, 1)
#             ut = np.swapaxes(ut, 1, 2)

#             rf = tifffile.imread(rfs[i])
#             rf = np.swapaxes(rf, 0, 1)
#             rf = np.swapaxes(rf, 1, 2)

#             if xct_paddeds[i] == None: #if is not already padded, load the original xct

#                 xct_folder = Path(frontals[i])

#                 xct_output_folder = xct_folder.parent

#                 xct = tifffile.imread(frontals[i])
#                 xct = np.swapaxes(xct, 0, 1)
#                 xct = np.swapaxes(xct, 1, 2)
            
#             else: #if padded, load the padded xct

#                 xct_folder = Path(xct_paddeds[i])

#                 xct_output_folder = xct_folder.parent

#                 xct = read_sequence(xct_paddeds[i])
#                 xct = np.swapaxes(xct, 0, 1)
#                 xct = np.swapaxes(xct, 1, 2)

#             #check if xct is padded or not
#             if xct_paddeds[i] == None: #It is not padded

#                 registered,xct_padded = reg.main(ut,rf,xct,False)

#                 tifffile.imwrite(ut_output_file, registered)

#                 xct_padded = np.swapaxes(xct_padded, 1, 2)
#                 xct_padded = np.swapaxes(xct_padded, 0, 1)

#                 write_sequence(xct_output_folder, 'padded', xct_padded)

#                 registered_uts[i] = str(ut_output_file)

#                 xct_paddeds[i] = str(xct_output_folder / 'padded')
            
#             else:

#                 registered = reg.main(ut,rf,xct,True)

#                 tifffile.imwrite(ut_output_file, registered)

#                 registered_uts[i] = str(ut_output_file)

        
#         except Exception as e:
#             print(f'Error processing {ids[i]}')
#             print(e)
    
#     if registered_uts_lente[i] == None:

#         try:

#             print(f'Registering {ids[i]} lens')

#             ut_file = str(rfs_lente[i])

#             ut_output_file = Path(ut_file[:-4] + '_registered.tif')

#             ut = tifffile.imread(amps_lente[i])
#             ut = np.swapaxes(ut, 0, 1)
#             ut = np.swapaxes(ut, 1, 2)

#             rf = tifffile.imread(rfs_lente[i])
#             rf = np.swapaxes(rf, 0, 1)
#             rf = np.swapaxes(rf, 1, 2)

#             if xct_paddeds[i] == None: #if is not already padded, load the original xct

#                 xct_folder = Path(frontals[i])

#                 xct_output_folder = xct_folder.parent

#                 xct = tifffile.imread(frontals[i])
#                 xct = np.swapaxes(xct, 0, 1)
#                 xct = np.swapaxes(xct, 1, 2)
            
#             else: #if padded, load the padded xct

#                 xct_folder = Path(xct_paddeds[i])

#                 xct_output_folder = xct_folder.parent

#                 xct = read_sequence(xct_paddeds[i])
#                 xct = np.swapaxes(xct, 0, 1)
#                 xct = np.swapaxes(xct, 1, 2)

#             #check if xct is padded or not
#             if xct_paddeds[i] == None: #It is not padded

#                 registered,xct_padded = reg.main(ut,rf,xct,False)

#                 tifffile.imwrite(ut_output_file, registered)

#                 xct_padded = np.swapaxes(xct_padded, 1, 2)
#                 xct_padded = np.swapaxes(xct_padded, 0, 1)

#                 write_sequence(xct_output_folder, 'padded', xct_padded)

#                 registered_uts_lente[i] = str(ut_output_file)

#                 xct_paddeds[i] = str(xct_output_folder / 'padded')
            
#             else:

#                 registered = reg.main(ut,rf,xct,True)

#                 tifffile.imwrite(ut_output_file, registered)

#                 registered_uts_lente[i] = str(ut_output_file)
        
#         except Exception as e:
#             print(f'Error processing {ids[i]}')
#             print(e)

In [7]:
for i in range(len(ids)):

    #utvsxct register
    if registration_parameters[i] == None: #if is not already registered

        try:

            print(f'Registering {ids[i]}')

            ut_file = str(rfs_og[i])

            ut_output_file = Path(ut_file[:-4] + '_registered.tif')

            rf = tifffile.imread(rfs_og[i])
            amp = alg.hillbert_transform(rf)
            gate = alg.auto_gate(amp)
            # Align the amplitude and the volume based on the maximum value in the gated range
            amp, _ = alg.double_align(amp,rf, gate)
            amp = np.swapaxes(amp, 0, 1)
            amp = np.swapaxes(amp, 1, 2)

            xct = tifffile.imread(frontals[i])
            xct = np.swapaxes(xct, 0, 1)
            xct = np.swapaxes(xct, 1, 2)

            parameters = reg.main_2(amp,xct)

            registration_parameters[i] = parameters

        except Exception as e:
            print(f'Error processing {ids[i]}')
            print(e)
    
    if registration_parameters_lente[i] == None:

        try:

            print(f'Registering {ids[i]} lens')

            ut_file = str(rfs_og_lente[i])

            ut_output_file = Path(ut_file[:-4] + '_registered.tif')

            rf = tifffile.imread(rfs_og_lente[i])
            amp = alg.hillbert_transform(rf)
            gate = alg.auto_gate(amp)
            # Align the amplitude and the volume based on the maximum value in the gated range
            amp, _ = alg.double_align(amp,rf, gate)
            amp = np.swapaxes(amp, 0, 1)
            amp = np.swapaxes(amp, 1, 2)

            xct = tifffile.imread(frontals[i])
            xct = np.swapaxes(xct, 0, 1)
            xct = np.swapaxes(xct, 1, 2)

            parameters = reg.main_2(amp,xct)

            registration_parameters_lente[i] = parameters
        
        except Exception as e:
            print(f'Error processing {ids[i]}')
            print(e)

Registering 1_26 lens
Preprocessing
Preprocessed
Registering
Registered
Registering 1_27 lens
Preprocessing
Preprocessed
Registering
Registered
Registering 1_28 lens
Preprocessing
Preprocessed
Registering
Registered
Registering 1_29 lens
Preprocessing
Preprocessed
Registering
Registered
Registering 1_21 lens
Preprocessing
Preprocessed
Registering
Registered
Registering 1_22 lens
Preprocessing
Preprocessed
Registering
Registered
Registering 1_23 lens
Preprocessing
Preprocessed
Registering
Registered
Registering 1_24 lens
Preprocessing
Preprocessed
Registering
Registered
Registering 1_31 lens
Preprocessing
Preprocessed
Registering
Registered
Registering 1_32 lens
Preprocessing
Preprocessed
Registering
Registered
Registering 1_33 lens
Preprocessing
Preprocessed
Registering
Registered
Registering 1_35 lens
Preprocessing
Preprocessed
Registering
Registered
Registering 1_6 lens
Preprocessing
Preprocessed
Registering
Registered
Registering 1_7 lens
Preprocessing
Preprocessed
Registering
Regis

In [8]:
#update database

database['UT registered'] = registered_uts

database['UT registered lente'] = registered_uts_lente

database['XCT padded'] = xct_paddeds

database['Registration Parameters'] = registration_parameters

database['Registration Parameters lente'] = registration_parameters_lente

In [9]:
database.to_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv', index=False)

# Loading parameters

In [10]:
database = pd.read_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv')

database = database.replace({np.nan: None})

parameters = database['Registration Parameters'].values

a = parameters[0]

a

'[[ 9.99746705e-01 -2.25061462e-02 -7.11426361e+02]\n [ 2.25061462e-02  9.99746705e-01 -1.99841467e+02]\n [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]'

In [11]:
def to_matrix(string):

    matrix1 = float(string[2:17])

    matrix2 = float(string[17:33])

    matrix3 = float(string[33:49])

    matrix4 = float(string[53:68])

    matrix5 = float(string[68:84])

    matrix6 = float(string[84:100])

    matrix7 = float(string[105:121])

    matrix8 = float(string[121:137])

    matrix9 = float(string[137:151])

    matrix = np.array([[matrix1,matrix2,matrix3],[matrix4,matrix5,matrix6],[matrix7,matrix8,matrix9]])

    return matrix

In [12]:
to_matrix(a)

array([[ 9.99746705e-01, -2.25061462e-02, -7.11426361e+02],
       [ 2.25061462e-02,  9.99746705e-01, -1.99841467e+02],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])